In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
 
import numpy as np # linear algebra
import os
import timeit
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
np.random.seed(12049)

In [4]:
def get_plot_loss(model, model_name):
    fig = plt.figure()
    plt.subplot(2, 1, 1)
    plt.plot(model.history.history["loss"])
    plt.plot(model.history.history["val_loss"])
    plt.title(f"{model_name} \n\n Model Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Valid"], loc="upper right")

    plt.subplot(2, 1, 2)
    plt.plot(model.history.history["mae"])
    plt.plot(model.history.history["val_mae"])
    plt.title("Model Mean Absolute Error")
    plt.ylabel("Mean Absolute Error")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Valid"], loc="upper right")

    plt.tight_layout()
    plt.show()


def get_evaluate(data, name, model):
    score_model = model.evaluate(data, verbose=1)
    print(f"{name} loss: {score_model:.2f}")


def get_predict(data, model):
    predict_model = model.predict(data)
    return predict_model


def get_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    print(f"Mean Absolute Error - {model_name}: {mae:.2f}")
    print(f"Mean Squared Error - {model_name}: {mse:.2f}")


In [11]:
# Load data
base_dir = "/kaggle/input/rsna-bone-age"
train_path = os.path.join(base_dir, 'boneage-training-dataset')
test_path = os.path.join(base_dir, 'boneage-test-dataset')

In [6]:
model_name = "ResNet50_regression"
target_size = (224, 224)
epochs = 100
batch_size = 32
img_shape = (224, 224, 3)


In [7]:
aug_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [12]:
train_generator = aug_datagen.flow_from_directory(
    train_path, target_size=target_size, batch_size=batch_size, shuffle=True
)

Found 12611 images belonging to 1 classes.


In [13]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255
)

In [14]:
test_generator = test_datagen.flow_from_directory(
    test_path, target_size=target_size, batch_size=batch_size, shuffle=False
)

Found 200 images belonging to 1 classes.


In [15]:
early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.01, patience=8, restore_best_weights=True
)

In [16]:
plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss', factor=0.1, min_delta=0.01, min_lr=1e-10, patience=4, mode='auto'
)

In [30]:
# Load pre-trained ResNet50 model
base_model = tf.keras.applications.ResNet50(
    input_shape=img_shape, include_top=False, weights=None
)
weights_path = '/kaggle/input/resnet50-weights-h5/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model.load_weights(weights_path)

In [37]:
for layer in base_model.layers:
    layer.trainable = True

model_regression = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='linear')
])
model_regression.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23589761 (89.99 MB)
Trainable params: 23536641 (89.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [39]:
model_regression.compile(
    optimizer='adam', loss='mean_squared_error', metrics=['mae']
)
start_time = timeit.default_timer()
history = model_regression.fit(
    train_generator, epochs=epochs, batch_size=batch_size, callbacks=[early, plateau],
    validation_data=test_generator, verbose=1
)

stop_time = timeit.default_timer()

Epoch 1/100
395/395 [==============================] - 3609s 9s/step - loss: 0.1294 - mae: 0.1524 - val_loss: 0.0096 - val_mae: 0.0977 - lr: 0.0010
Epoch 2/100
395/395 [==============================] - 3567s 9s/step - loss: 0.0038 - mae: 0.0479 - val_loss: 0.0101 - val_mae: 0.0995 - lr: 0.0010
Epoch 3/100
395/395 [==============================] - 3620s 9s/step - loss: 0.0017 - mae: 0.0329 - val_loss: 0.0017 - val_mae: 0.0377 - lr: 0.0010
Epoch 4/100
395/395 [==============================] - 3642s 9s/step - loss: 0.0011 - mae: 0.0265 - val_loss: 2.9339e-04 - val_mae: 0.0147 - lr: 0.0010
Epoch 5/100
395/395 [==============================] - 3527s 9s/step - loss: 8.9544e-04 - mae: 0.0238 - val_loss: 8.9247e-05 - val_mae: 0.0078 - lr: 0.0010
Epoch 6/100
395/395 [==============================] - 3626s 9s/step - loss: 7.8716e-04 - mae: 0.0223 - val_loss: 1.1206e-04 - val_mae: 0.0085 - lr: 0.0010
Epoch 7/100
395/395 [==============================] - 3778s 10s/step - loss: 5.9430e-04 - m

In [33]:
execution_time = (stop_time - start_time) / 60.0
print(f"Model {model_name} fine tuning executed in {execution_time:.2f} minutes")


In [34]:
# Plot loss and mean absolute error
def get_plot_loss(model, model_name):
    plt.plot(model.history.history["loss"])
    plt.plot(model.history.history["val_loss"])
    plt.title(f"{model_name} \n\n Model Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Valid"], loc="upper right")

    plt.show()

get_plot_loss(history, model_name)

In [ ]:
# Evaluate the model
def get_evaluate(data, name, model):
    score_model = model.evaluate(data, verbose=1)
    print(f"{name} loss: {score_model:.2f}")

get_evaluate(test_generator, "Test", model_regression)

In [ ]:
def get_predict(data, model):
    predict_model = model.predict(data)
    return predict_model

y_pred = get_predict(test_generator, model_regression)

In [ ]:
def get_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    print(f"Mean Absolute Error - {model_name}: {mae:.2f}")
    print(f"Mean Squared Error - {model_name}: {mse:.2f}")

get_metrics(test_generator.labels, y_pred, model_name)